# IMDb

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/IMDB_Logo_2016.svg/640px-IMDB_Logo_2016.svg.png?1623833316814">

## Descriptions

    - IMDb (an acronym for Internet Movie Database) is an online database of information related to films, television programs, home videos, video games, and streaming content online – including cast, production crew and personal biographies, plot summaries, trivia, ratings, and fan and critical reviews. An additional fan feature, message boards, was abandoned in February 2017. Originally a fan-operated website, the database is now owned and operated by IMDb.com, Inc., a subsidiary of Amazon.

    - As of December 2020, IMDb has approximately 7.5 million titles (including episodes) and 10.4 million personalities in its database, as well as 83 million registered users.

    - IMDb began as a movie database on the Usenet group "rec.arts.movies" in 1990 and moved to the web in 1993.

    - Information ref: https://www.wikiwand.com/en/IMDb

    - Now, we're interested in what's going on and finding insight by given data from "https://relational.fit.cvut.cz/dataset/IMDb"

## Objective
- Building movies recommendation based on personal information from user. 

**NOTED that we haven't currently planned to build model yet
`
In this project, we only prepare data for modeling**

## Proceduces for this project

    1 Business Understanding
    2 Data Preparation
        - Collecting
        - Cleaning
    3 Data Understanding
    4 Describing Data
        - EDA

## Additional Libraries
- pymysql
- pandas_profiling

In [0]:
# ! pip install pymysql

In [0]:
# ! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 

## Entity Relational Diagram

    A Diagram display the relations between databases is shown here

<img src="https://relational.fit.cvut.cz/assets/img/datasets-generated/imdb_ijs.svg">

<img src="https://media1.tenor.com/images/871791f2c3b6a691428d5eb69001647e/tenor.gif?itemid=16088453" width="350" height="350">

## Data Preparation section

### Collecting data

- Retrieve tables from given database

In [1]:
# Inspects all all tables, are still available, from database

import pymysql.cursors

hostname =  "relational.fit.cvut.cz"
port = 3306

username = "guest"
password = "relational"
db = "imdb_ijs"

connections = pymysql.connect(
    host=hostname,
    port=port,
    user=username,
    password=password,
    db=db,
    charset="utf8mb4",
    cursorclass=pymysql.cursors.DictCursor
)

cursor = connections.cursor()
cursor.execute("show tables;")
tables = cursor.fetchall()
cursor.close()

for ele in tables:
    print(ele)

{'Tables_in_imdb_ijs': 'actors'}
{'Tables_in_imdb_ijs': 'directors'}
{'Tables_in_imdb_ijs': 'directors_genres'}
{'Tables_in_imdb_ijs': 'movies'}
{'Tables_in_imdb_ijs': 'movies_directors'}
{'Tables_in_imdb_ijs': 'movies_genres'}
{'Tables_in_imdb_ijs': 'roles'}


In [2]:
import pandas as pd

In [3]:
# Import(1)
# Extracts tables from database to pandas dataframe

def extract_dataframe(table_name: str):
    with connections.cursor() as cursor:
        # select all from given table name without any conditions
        select = "SELECT * FROM " + table_name
        cursor.execute(select)
        result = cursor.fetchall()
        return pd.DataFrame(result)

raw_actors = extract_dataframe("actors")
raw_directors = extract_dataframe("directors")
raw_directorsGen = extract_dataframe("directors_genres")
raw_movies = extract_dataframe("movies")
raw_moviesDirec = extract_dataframe("movies_directors")
raw_moviesGen = extract_dataframe("movies_genres")
raw_roles = extract_dataframe("roles")

In [0]:
# Import (2)
# you can change paths later

raw_actors = pd.read_csv("actors.csv")
raw_directors = pd.read_csv("directors.csv")
raw_directorsGen = pd.read_csv("directors_genres.csv")
raw_movies = pd.read_csv("movies.csv")
raw_moviesDirec = pd.read_csv("movies_directors.csv")
raw_moviesGen = pd.read_csv("movies_genres.csv")
raw_roles = pd.read_csv("roles.csv")

In [4]:
# Assigning name

dfs_name = {
    "Actors" : raw_actors,
    "Directors" : raw_directors,
    "Directors-genres" : raw_directorsGen,
    "Movies" : raw_movies,
    "Movies-directors" : raw_moviesDirec,
    "Movies-genres" : raw_moviesGen,
    "Roles" : raw_roles 
}

In [5]:
# Exporting raw tables

raw_actors.to_csv("actors.csv")
raw_directors.to_csv("directors.csv")
raw_directorsGen.to_csv("directors_genres.csv")
raw_movies.to_csv("movies.csv")
raw_moviesDirec.to_csv("movies_directors.csv")
raw_moviesGen.to_csv("movies_genres.csv")
raw_roles.to_csv("roles.csv")

In [0]:
# Print the first few row of each

for name_df, value in dfs_name.items():
    print(name_df)
    print(value.head())
    print()

### Data Cleaning
    1 Validate each columns are in the right type
    2 Remove missing values
    3 Adjust values error (the value that makes no sense)
    4 Remove duplicates rows
    5 Standardize data (if possible)

<img src="https://media1.tenor.com/images/5237c33065a69a734309974a8b7f029b/tenor.gif?itemid=15522288" width="200" height="200">

#### Correcting data types & Finding missing values
- Inspect all data type in each columns that has been stored correctly based on **Data Dictionary**
- Indentify the number of missing values

In [0]:
# Looking basic information of each tables
# The basic information can tell us the type of each columns, null object

for name_df, df in dfs_name.items():
    print(name_df)
    print(df.info())
    print()

##### Report

**Correcting data types**

- We inspected that **all columns are in the right type**
- e.g. all id columns are in integer type, rank is in float type, names or genres are in object typr

**Missing values**

- There are massively null valus in Movies dataframe (321024)

#### Remove missing values

- Remove on [ movies table ]

In [0]:
# Remove row that contains missing value

droppedNull_movies = raw_movies.dropna()

droppedNull_movies =  droppedNull_movies.reset_index(drop=True)

# Update dictionary
dfs_name["Movies"] = droppedNull_movies

droppedNull_movies.head(10)

In [0]:
# check all id column of each table are unique (no duplicate id)

ids_col = ["id", "director_id", "movie_id", "actor_id"]
for name, df in dfs_name.items():
    # find id column
    for id_col in df.columns:
        if id_col in ids_col:
            print(name, "on column", id_col)
            print("ID unique:",df[id_col].duplicated().any())
            print()
            break

##### Report

#### Adjust values error
- Check and change (if neccessary) the values whether are in the right format

In [0]:
# basic statistics
# Continue to check all dataframe

droppedNull_movies.describe()

##### Result

**Movies dataframe**

- We ignore id column (already checked id column)
- Year initially collected since 1890 to 2004 -> (make sense)
- Rank (rating) min is 1 through 9.9 (max) -> (make sense)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=056bc3c2-85b4-467d-bb7f-c23e60e46336' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>